In [1]:
from deepface import DeepFace
import matplotlib.pyplot as plt
import cv2
import os
import pandas

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

Test execution for every backend of deepface.

In [3]:
detectors_backends = [
    'opencv',
    'ssd',
    'dlib',
    'mtcnn',
    'retinaface',
    'mediapipe'
]

def analyze_performance(backend, path, count_false_positive = False):
    fail_counter = 0
    confidence_sum = 0
    
    for i, img_path in enumerate(os.listdir(path)):
            try:
                img_path_full = path + "/" + img_path
                face_obj = DeepFace.extract_faces(img_path_full, target_size=(224, 224), detector_backend=backend)
                
                if not count_false_positive:
                    if len(face_obj) > 1:
                        fail_counter += 1
                    else:
                        confidence_sum += face_obj[0]['confidence']
                                 
            except:
                fail_counter += 1
            
    return fail_counter, len(os.listdir(path)), confidence_sum

(1708, 2560, 3)


### Test the big faces dataset for every backend

In [9]:
big_images_perf = dict()

for backend in detectors_backends:
       print("Running: ", backend)
       entry = dict()
       fail_counter, dataset_len, confidence_sum = analyze_performance(backend, "../dataset/big")
       fail_rate = fail_counter / dataset_len
       entry['fails'] = fail_counter
       entry['fail rate'] = fail_rate
       entry['mean confidence'] = confidence_sum / (dataset_len - fail_counter)
       big_images_perf[backend] = entry

Running:  opencv
Running:  ssd
Running:  dlib
Running:  mtcnn
6/6 [==============================] - 0s 9ms/step
Running:  retinaface
Running:  mediapipe


In [10]:
pandas.DataFrame(big_images_perf)

,opencv,ssd,dlib,mtcnn,retinaface,mediapipe
fails,9.000000,5.000000,0.000000,6.000000,1.000000,4.000000
fail rate,0.290323,0.161290,0.000000,0.193548,0.032258,0.129032
mean confidence,8.133990,0.987383,1.900457,0.999779,0.996826,0.834149


### Test odd faces for every backend

In [6]:
odd_images_perf = dict()

for backend in detectors_backends:
       print("Running: ", backend)
       entry = dict()
       fail_counter, dataset_len, confidence_sum = analyze_performance(backend, "../dataset/notnormal")
       fail_rate = fail_counter / dataset_len
       entry['fails'] = fail_counter
       entry['fail rate'] = fail_rate
       entry['mean confidence'] = confidence_sum / (dataset_len - fail_counter)
       odd_images_perf[backend] = entry

Running:  opencv
Running:  ssd
Running:  dlib
Running:  mtcnn
20/20 [==============================] - 0s 14ms/step
Running:  retinaface
Running:  mediapipe


In [7]:
pandas.DataFrame(odd_images_perf)

,opencv,ssd,dlib,mtcnn,retinaface,mediapipe
fails,14.000000,4.000000,7.000000,6.000000,2.000000,13.000000
fail rate,0.466667,0.133333,0.233333,0.200000,0.066667,0.433333
mean confidence,5.607155,0.986816,1.264801,0.993381,0.998463,0.770538
